# Ex19

Por volta de 435 a.C., Esparta decidiu convocar reservistas para suplementar seu exército regular. Os novos combatentes podiam ser alistados por 1, 2, 3 ou 4 anos, respectivamente com custos de $10, $18, $25 e $31. A força mínima total de combatantes reservistas foi estabelecida na tabela abaixo. Como um general espartano, você poderia achar uma política de alistamento ótima para os próximos 10 anos, resolvendo o problema com um modelo de otimização. Formule o modelo e resolva.

| Ano    | Reservistas |
|--------|-------------|
| 435 AC | 10.200      |
| 434 AC | 11.800      |
| 433 AC | 10.500      |
| 432 AC | 12.300      |
| 431 AC | 15.000      |
| 430 AC | 14.500      |
| 429 AC | 14.000      |
| 428 AC | 16.100      |
| 427 AC | 16.400      |
| 426 AC | 16.500      |

Considere que não será possível contratar mais do que 13.000 reservistas em um único ano. Qual seria a solução?







## **Definição das Variáveis**
Sejam as variáveis de decisão:

- $x_{ti}$ = Número de reservistas alistados no **ano** $t$ para **$i$ anos** de serviço, onde:
  - $t$ representa o ano dentro do período analisado ($1 \leq t \leq 10$).
  - $i$ indica o tempo de serviço dos reservistas ($1 \leq i \leq 4$).



## **Função Objetivo**
Queremos **minimizar o custo total de alistamento**, dado por:

$$
\min \sum_{t=1}^{10} (10x_{t1} + 18x_{t2} + 25x_{t3} + 31x_{t4})
$$

onde:
- Os custos de alistamento por reservista são **$10, $18, $25 e $31** conforme os **anos de serviço**.



## **Restrições**
### **1. Exigência mínima de reservistas em cada ano**
A força mínima deve ser respeitada conforme a tabela de demanda:

$$
x_{t1} + x_{(t-1)2} + x_{(t-2)3} + x_{(t-3)4} \geq R_t, \quad \forall t = 1, 2, ..., 10
$$

onde:
- $R_t$ representa o número mínimo de reservistas exigidos no ano $t$.
- Os reservistas recrutados nos anos anteriores continuam servindo de acordo com seu tempo de serviço.

### **2. Limite máximo de recrutamento por ano**
Não podemos alistar mais do que **13.000** reservistas por ano:

$$
x_{t1} + x_{t2} + x_{t3} + x_{t4} \leq 13000, \quad \forall t = 1, 2, ..., 10
$$

### **3. Restrição de não negatividade**
As variáveis de decisão devem ser **não negativas**:

$$
x_{ti} \geq 0, \quad \forall t, i
$$






In [1]:
import pyomo.environ as pyo

In [2]:
# Criação do modelo
model = pyo.ConcreteModel()

In [3]:

# Conjuntos
model.T = pyo.RangeSet(1, 10) 
model.I = pyo.RangeSet(1, 4)   


In [4]:
# Custo de alistamento por tempo de serviço
custo_alistamento = {1: 10, 2: 18, 3: 25, 4: 31}
model.custo = pyo.Param(model.I, initialize=custo_alistamento)

# Demanda de reservistas por ano
demanda_reservistas = {1: 10200, 2: 11800, 3: 10500, 4: 12300, 5: 15000,
                       6: 14500, 7: 14000, 8: 16100, 9: 16400, 10: 16500}
model.demanda = pyo.Param(model.T, initialize=demanda_reservistas)

# Capacidade máxima de recrutamento por ano
capacidade_recrutamento = 13000

In [5]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------

model.x = pyo.Var(model.T, model.I, domain=pyo.NonNegativeIntegers)

In [6]:

# ------------------------------
# Função Objetivo
# ------------------------------
def fo(model):
    return sum(model.x[t, i] * model.custo[i] for t in model.T for i in model.I)

model.objetivo = pyo.Objective(rule=fo, sense=pyo.minimize)


In [7]:
# ------------------------------
# Restrições
# ------------------------------

# 1. Exigência mínima de reservistas em cada ano
def restricao_demanda(model, t):
    a = 0
    if t >= 1:
        a += model.x[t, 1]
    if t-1 >= 1:
        a += model.x[t-1, 2]
    if t-2 >= 1:
        a += model.x[t-2, 3]
    if t-3 >= 1:
        a += model.x[t-3, 4]
    return a >= model.demanda[t]

model.restricao_demanda = pyo.Constraint(model.T, rule=restricao_demanda)

# 2. Limite máximo de recrutamento por ano
def restricao_max_recrutamento(model, t):
    return sum(model.x[t, i] for i in model.I) <= capacidade_recrutamento

model.restricao_max_recrutamento = pyo.Constraint(model.T, rule=restricao_max_recrutamento)


In [8]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex19.lp", io_options={"symbolic_solver_labels": True})

('ex19.lp', 1747183686128)

In [9]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.objetivo)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)

MIP  has 20 rows; 40 cols; 74 nonzeros; 40 integer variables (0 binary)
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e+01, 3e+01]
  Bound  [0e+00, 0e+00]
  RHS    [1e+04, 2e+04]
Presolving model
19 rows, 33 cols, 66 nonzeros  0s
16 rows, 31 cols, 61 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   16 rows
   31 cols (0 binary, 31 integer, 0 implied int., 0 continuous)
   61 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0     

RuntimeError: A feasible solution was not found, so no solution can be loaded.Please set opt.config.load_solution=False and check results.termination_condition and results.best_feasible_objective before loading a solution.

MODELO INFACTIVEL

In [100]:
for t in model.T:
    for i in model.I:
        
        valor = pyo.value(model.x[t, i])
        if valor > 0:
            print(f"Ano {t}, Tempo de Serviço {i} anos: {valor:.2f} reservistas")

ERROR: evaluating object as numeric value: x[1,1]
        (object: <class 'pyomo.core.base.var._GeneralVarData'>)
    No value for uninitialized NumericValue object x[1,1]


ValueError: No value for uninitialized NumericValue object x[1,1]